In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/eng-san/san_train.csv
/kaggle/input/eng-san/san_valid.csv
/kaggle/input/eng-san/san_test.csv
/kaggle/input/eng-hin/hin_valid.csv
/kaggle/input/eng-hin/hin_test.csv
/kaggle/input/eng-hin/hin_train.csv


In [2]:
# import cell
%matplotlib inline
import random

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader
from torch.utils.data import TensorDataset

import time
import math

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

SOS_token = "@"
EOS_token = "#"
PAD_token = "^"
UNK_token = "$"

SOS_idx = 0
EOS_idx = 1
PAD_idx = 2
UNK_idx = 3

cuda


In [4]:
def timeInMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    s = format(s, ".0f")
    return str(m) + "m " + str(s) + "s"

In [5]:
class Script:
    def __init__(self, name):
        self.name = name
        self.char2index = {SOS_token: SOS_idx, EOS_token: EOS_idx, PAD_token: PAD_idx, UNK_token: UNK_idx}
        self.char2count = {}
        self.index2char = {SOS_idx: SOS_token, EOS_idx: EOS_token, PAD_idx: PAD_token, UNK_idx: UNK_token}
        self.n_chars = 4  # Count SOS, EOS, PAD and UNK

    def addWord(self, word):
        for char in word:
            self.addChar(char)

    def addChar(self, char):
        if char not in self.char2index:
            self.char2index[char] = self.n_chars
            self.char2count[char] = 1
            self.index2char[self.n_chars] = char
            self.n_chars += 1
        else:
            self.char2count[char] += 1

In [6]:
def prepareVocab(data, in_scr="lat", out_scr="dev"):
    input_vocab = Script(in_scr)
    output_vocab = Script(out_scr)
    
    for pair in data:
        input_vocab.addWord(pair[0])
        output_vocab.addWord(pair[1])
    
    return input_vocab, output_vocab

In [7]:
def tensorFromWord(word, vocab, sos=False, eos=False):
    char_list = []
    if sos:
        char_list.append(vocab.char2index[SOS_token])
    for char in word:
        if char in vocab.char2index:
            char_list.append(vocab.char2index[char])
        else:
            char_list.append(vocab.char2index[UNK_token])
    if eos:
        char_list.append(vocab.char2index[EOS_token])
    char_tensor = torch.tensor(char_list, dtype=torch.long)
    return char_tensor

In [8]:
def processData(data, vocab, sos=False, eos=False):
    tensor_list = []
    for word in data:
        word_tensor = tensorFromWord(word, vocab, sos, eos)
        tensor_list.append(word_tensor)
    word_tensor_pad = pad_sequence(tensor_list, padding_value=PAD_idx, batch_first=True)
    return word_tensor_pad

In [9]:
def wordFromTensor(word_tensor, vocab):
    word = ""
    for idx in word_tensor:
        if idx == EOS_idx:
            break
        if idx >= UNK_idx:
            word += vocab.index2char[idx.item()]
    return word

In [10]:
class Encoder(nn.Module):
    def __init__(self, cell_type, input_size, embedding_size, hidden_size, num_layers, dp):
        super(Encoder, self).__init__()
        self.cell_type = cell_type
        self.input_size = input_size
        self.embedding_size = embedding_size
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.dropout = nn.Dropout(dp)
        
        self.embedding = nn.Embedding(self.input_size, self.embedding_size)
        if self.cell_type == "RNN":
            self.cell = nn.RNN(self.embedding_size, self.hidden_size, self.num_layers, dropout=dp)
        elif self.cell_type == "GRU":
            self.cell = nn.GRU(self.embedding_size, self.hidden_size, self.num_layers, dropout=dp)
        elif self.cell_type == "LSTM":
            self.cell = nn.LSTM(self.embedding_size, self.hidden_size, self.num_layers, dropout=dp)

    def forward(self, x):
        # x shape: (seq_length, N) where N is batch size

        embedding = self.dropout(self.embedding(x))
        # embedding shape: (seq_length, N, embedding_size)
        
        cell = None
        if self.cell_type == "LSTM":
            outputs, (hidden, cell) = self.cell(embedding)
        else:
            outputs, hidden = self.cell(embedding)
        # outputs shape: (seq_length, N, hidden_size)

        return hidden, cell

In [11]:
class Decoder(nn.Module):
    def __init__(
        self, cell_type, input_size, embedding_size, hidden_size, output_size, num_layers, dp
    ):
        super(Decoder, self).__init__()
        self.cell_type = cell_type
        self.input_size = input_size
        self.embedding_size = embedding_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.num_layers = num_layers
        self.dropout = nn.Dropout(dp)

        self.embedding = nn.Embedding(self.input_size, self.embedding_size)
        if self.cell_type == "RNN":
            self.cell = nn.RNN(self.embedding_size, self.hidden_size, self.num_layers, dropout=dp)
        elif self.cell_type == "GRU":
            self.cell = nn.GRU(self.embedding_size, self.hidden_size, self.num_layers, dropout=dp)
        elif self.cell_type == "LSTM":
            self.cell = nn.LSTM(self.embedding_size, self.hidden_size, self.num_layers, dropout=dp)
        self.fc = nn.Linear(self.hidden_size, self.output_size)

    def forward(self, x, hidden, cell):
        # x shape: (N) where N is for batch size, we want it to be (1, N), seq_length
        # is 1 here because we are sending in a single word and not a sentence
        x = x.unsqueeze(0)

        embedding = self.dropout(self.embedding(x))
        # embedding shape: (1, N, embedding_size)

        if self.cell_type == "LSTM":
            outputs, (hidden, cell) = self.cell(embedding, (hidden, cell))
        else:
            outputs, hidden = self.cell(embedding, hidden)
        # outputs shape: (1, N, hidden_size)

        predictions = self.fc(outputs)

        # predictions shape: (1, N, length_target_vocabulary) to send it to
        # loss function we want it to be (N, length_target_vocabulary) so we're
        # just gonna remove the first dim
        predictions = predictions.squeeze(0)

        return predictions, hidden, cell

In [12]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder):
        super(Seq2Seq, self).__init__()
        self.encoder = encoder
        self.decoder = decoder

    def forward(self, source, target, teacher_force_ratio=0.5):
        batch_sz = source.shape[1]
        target_len = target.shape[0]
        target_vocab_size = self.decoder.output_size

        outputs = torch.zeros(target_len, batch_sz, target_vocab_size).to(device)

        hidden, cell = self.encoder(source)

        # Grab the first input to the Decoder which will be <SOS> token
        x = target[0]

        for t in range(1, target_len):
            # Use previous hidden, cell as context from encoder at start
            output, hidden, cell = self.decoder(x, hidden, cell)

            # Store next output prediction
            outputs[t] = output

            # Get the best word the Decoder predicted (index in the vocabulary)
            best_guess = output.argmax(dim=1)

            # With probability of teacher_force_ratio we take the actual next word
            # otherwise we take the word that the Decoder predicted it to be.
            # Teacher Forcing is used so that the model gets used to seeing
            # similar inputs at training and testing time, if teacher forcing is 1
            # then inputs at test time might be completely different than what the
            # network is used to. This was a long comment.
            x = target[t] if random.random() < teacher_force_ratio else best_guess

        return outputs

In [13]:
def sum_accuracy(preds, target):
    num_equal_columns = torch.logical_or(preds == target, target == PAD_idx).all(dim=0).sum().item()
    return num_equal_columns

In [14]:
# load dataset
train_data = pd.read_csv('/kaggle/input/eng-hin/hin_train.csv', sep=',', header=None).values
test_data = pd.read_csv('/kaggle/input/eng-hin/hin_test.csv', sep=',', header=None).values
valid_data = pd.read_csv('/kaggle/input/eng-hin/hin_valid.csv', sep=',', header=None).values

In [15]:
# build vocabulary
x_vocab, y_vocab = prepareVocab(train_data)

In [16]:
print(x_vocab.n_chars)
print(y_vocab.n_chars)

30
68


In [17]:
x_train = processData(train_data[:,0], x_vocab, eos=True).to(device=device)
x_test = processData(test_data[:,0], x_vocab, eos=True).to(device=device)
x_valid = processData(valid_data[:,0], x_vocab, eos=True).to(device=device)

y_train = processData(train_data[:,1], y_vocab, sos=True, eos=True).to(device=device)
y_test = processData(test_data[:,1], y_vocab, sos=True, eos=True).to(device=device)
y_valid = processData(valid_data[:,1], y_vocab, sos=True, eos=True).to(device=device)

In [18]:
n_train = x_train.size(0)
n_valid = x_valid.size(0)
n_test = x_test.size(0)

print(n_train, n_valid, n_test)

51200 4096 4096


In [19]:
train_dataset = TensorDataset(x_train, y_train)
train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True)

valid_dataset = TensorDataset(x_valid, y_valid)
valid_dataloader = DataLoader(valid_dataset, batch_size=64)

In [30]:
# Training hyperparameters
num_epochs = 5
learning_rate = 0.001
batch_size = 64

# Fixed parameters for encoder and decoder
input_size_encoder = x_vocab.n_chars
input_size_decoder = y_vocab.n_chars
output_size = input_size_decoder

# Model hyperparameters
cell_type = "GRU"
embedding_size = 256
hidden_size = 256  # Needs to be the same for both RNN's
num_layers = 2
dropout = 0.0

In [31]:
encoder_net = Encoder(
    cell_type, input_size_encoder, embedding_size, hidden_size, num_layers, dropout
).to(device)

decoder_net = Decoder(
    cell_type,
    input_size_decoder,
    embedding_size,
    hidden_size,
    output_size,
    num_layers,
    dropout,
).to(device)

model = Seq2Seq(encoder_net, decoder_net).to(device)
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss()

In [32]:
start = time.time()
for epoch in range(num_epochs):
    print(f"[Epoch {epoch+1} / {num_epochs}]")

    model.train()
    
    for batch_idx, (input_seq, target_seq) in enumerate(train_dataloader):
        # Get input and targets and get to cuda
        input_seq = input_seq.T.to(device)
        target_seq = target_seq.T.to(device)

        # Forward prop
        output = model(input_seq, target_seq)
        # Output is of shape (trg_len, batch_size, output_dim) but Cross Entropy Loss
        # doesn't take input in that form. For example if we have MNIST we want to have
        # output to be: (N, 10) and targets just (N). Here we can view it in a similar
        # way that we have output_words * batch_size that we want to send in into
        # our cost function, so we need to do some reshapin. While we're at it
        # Let's also remove the start token while we're at it
        output = output[1:].reshape(-1, output.shape[2])
        target = target_seq[1:].reshape(-1)

        optimizer.zero_grad()
        loss = criterion(output, target)

        # Back prop
        loss.backward()

        # Clip to avoid exploding gradient issues, makes sure grads are
        # within a healthy range
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1)

        # Gradient descent step
        optimizer.step()
        
    #-----------------------------------------------
    # Train loss and accuracy
    model.eval()
    
    train_loss_epoch = 0
    n_correct_train = 0
    
    with torch.no_grad():
        for batch_idx, (input_seq, target_seq) in enumerate(train_dataloader):
            # Get input and targets and get to cuda
            input_seq = input_seq.T.to(device)
            target_seq = target_seq.T.to(device)

            # Forward prop
            output = model(input_seq, target_seq, teacher_force_ratio=0.0)
            
            pred_seq = output.argmax(dim=2)
            n_correct_train += sum_accuracy(pred_seq, target_seq)

            # Output is of shape (trg_len, batch_size, output_dim) but Cross Entropy Loss
            # doesn't take input in that form. For example if we have MNIST we want to have
            # output to be: (N, 10) and targets just (N). Here we can view it in a similar
            # way that we have output_words * batch_size that we want to send in into
            # our cost function, so we need to do some reshapin. While we're at it
            # Let's also remove the start token while we're at it
            output = output[1:].reshape(-1, output.shape[2])
            target = target_seq[1:].reshape(-1)
            
            loss = criterion(output, target)

            train_loss_epoch += loss.item()
        
        train_acc = n_correct_train / n_train
        train_loss_epoch /= len(train_dataloader)
        print(f"Training Loss: {train_loss_epoch:.2f}")
        print(f"Training Accuracy: {train_acc:.2f}")
    
    #-----------------------------------------------
    # Valid loss and accuracy
    model.eval()
    
    val_loss_epoch = 0
    n_correct_valid = 0
    
    with torch.no_grad():
        for batch_idx, (input_seq, target_seq) in enumerate(valid_dataloader):
            # Get input and targets and get to cuda
            input_seq = input_seq.T.to(device)
            target_seq = target_seq.T.to(device)

            # Forward prop
            output = model(input_seq, target_seq, teacher_force_ratio=0.0)
            
            pred_seq = output.argmax(dim=2)
            n_correct_valid += sum_accuracy(pred_seq, target_seq)

            # Output is of shape (trg_len, batch_size, output_dim) but Cross Entropy Loss
            # doesn't take input in that form. For example if we have MNIST we want to have
            # output to be: (N, 10) and targets just (N). Here we can view it in a similar
            # way that we have output_words * batch_size that we want to send in into
            # our cost function, so we need to do some reshapin. While we're at it
            # Let's also remove the start token while we're at it
            output = output[1:].reshape(-1, output.shape[2])
            target = target_seq[1:].reshape(-1)
            
            loss = criterion(output, target)

            val_loss_epoch += loss.item()
        
        valid_acc = n_correct_valid / n_valid
        val_loss_epoch /= len(valid_dataloader)
        print(f"Validation Loss: {val_loss_epoch:.2f}")
        print(f"Validation Accuracy: {valid_acc:.2f}")
    
    end = time.time()
    print("Time: ", timeInMinutes(end-start))
    print("----------------------------------")

[Epoch 1 / 5]
Training Loss: 0.82
Training Accuracy: 0.07
Validation Loss: 0.69
Validation Accuracy: 0.11
Time:  0m 36s
----------------------------------
[Epoch 2 / 5]
Training Loss: 0.68
Training Accuracy: 0.16
Validation Loss: 0.56
Validation Accuracy: 0.20
Time:  1m 12s
----------------------------------
[Epoch 3 / 5]
Training Loss: 0.62
Training Accuracy: 0.22
Validation Loss: 0.53
Validation Accuracy: 0.25
Time:  1m 48s
----------------------------------
[Epoch 4 / 5]
Training Loss: 0.58
Training Accuracy: 0.25
Validation Loss: 0.51
Validation Accuracy: 0.27
Time:  2m 24s
----------------------------------
[Epoch 5 / 5]
Training Loss: 0.58
Training Accuracy: 0.30
Validation Loss: 0.53
Validation Accuracy: 0.29
Time:  3m 0s
----------------------------------
